In [11]:
import h2o
h2o.init()
h2o.remove_all()

H2O cluster uptime:,8 days 12 hours 57 minutes 37 seconds 365 milliseconds
H2O cluster version:,3.8.2.3
H2O cluster name:,H2O_started_from_python_joe_igy224
H2O cluster total nodes:,1
H2O cluster total free memory:,631.7 MB
H2O cluster total cores:,1
H2O cluster allowed cores:,1
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321
H2O Connection proxy:,None


In [15]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
import numpy as np

df = h2o.import_file('FP_movies.psv')
print df.shape #(rows,columns)
train, test = df.split_frame([0.8], seed=63)
print len(train), len(test)


Parse Progress: [##################################################] 100%
(7071, 17)
5666 1405


In [16]:
train.head()

id,title,year,country,directorName,Actor1,Actor2,Actor3,rtAllCriticsRating,MovieStarPower,MovieStarPowerWCntry,AudienceRating,StarAud,Genre1,Genre2,Genre3,Actor1NameEndsInA
2,Jumanji,1995,USA,Joe Johnston,Robin Williams,Bonnie Hunt,Kirsten Dunst,5.6,159,52.47,3.2,3.7247,Children,Fantasy,Adventure,F
11,The American President,1995,USA,Rob Reiner,Michael Douglas,Annette Bening,Martin Sheen,7,138,45.54,3.2,3.6554,Romance,Comedy,Drama,F
18,Four Rooms,1995,USA,Alexandre Rockwell,Tim Roth,Valeria Golino,Jennifer Beals,3.5,82,27.06,3.5,3.7706,Thriller,Comedy,Drama,F
20,Money Train,1995,USA,Joseph Ruben,Wesley Snipes,Woody Harrelson,Jennifer Lopez,4,152,50.16,2.7,3.2016,Crime,Action,Thriller,F
27,Now and Then,1995,USA,Lesli Linka Glatter,Christina Ricci,Thora Birch,Gaby Hoffmann,4.7,78,25.74,3.5,3.7574,Drama,Drama,Drama,T
29,La cit� des enfants perdus,1995,France,Jean-Pierre Jeunet,Ron Perlman,Daniel Emilfork,Judith Vittet,7,35,32.9,4,4.329,Mystery,Fantasy,Sci-Fi,F
36,Dead Man Walking,1995,UK,Tim Robbins,Susan Sarandon,Sean Penn,Robert Prosky,8.2,156,140.4,3.6,5.004,Crime,Drama,Crime,F
43,Restoration,1995,USA,Michael Hoffman,Robert Downey Jr.,Polly Walker,Meg Ryan,6.3,143,47.19,3.3,3.7719,Drama,Drama,Drama,F
68,Gazon maudit,1995,France,Josiane Balasko,Victoria Abril,Josiane Balasko,Alain Chabat,6.1,31,29.14,3.4,3.6914,Romance,Comedy,Romance,T
77,Nico Icon,1995,Germany,Susanne Ofteringer,Andy Warhol,Billy Name,Jackson Browne,7.1,6,5.88,4,4.0588,Documentary,Documentary,Documentary,F


In [13]:
target_simple_Y = 'rtAllCriticsRating'
exp_multi_X = ['year','country','directorName','Actor1','MovieStarPowerWCntry','AudienceRating',
              'Genre1', 'Genre2', 'Genre3', 'Actor1NameEndsInA']

glm_movie_multi = H2OGeneralizedLinearEstimator(
                    model_id='glm_movie_multi',
                    family='gaussian', 
                    solver='IRLSM' 
                    ,lambda_search = True
                )  

glm_movie_multi.train(exp_multi_X, target_simple_Y, training_frame=train)
glm_movie_multi


glm Model Build Progress: [##################################################] 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  glm_movie_multi

GLM Model: summary



,family,link,regularization,lambda_search,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
,gaussian,identity,"Elastic Net (alpha = 0.5, lambda = 0.02004 )","nlambda = 100, lambda_max = 0.0, best_lambda = 0.02004",6544,18,0,py_14




ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 1.10550783849
R^2: 0.521622458762
Mean Residual Deviance: 1.10550783849
Null degrees of freedom: 5663
Residual degrees of freedom: 5645
Null deviance: 13089.2357133
Residual deviance: 6261.59639723
AIC: 16681.8621578

Scoring History: 


,timestamp,duration,iteration,lambda,predictors,explained_deviance_train,explained_deviance_test
,2016-05-10 07:34:42,0.000 sec,0,.2E1,1,-0.0000000,nan
,2016-05-10 07:34:42,0.022 sec,0,.19E1,2,0.0199077,nan
,2016-05-10 07:34:42,0.033 sec,0,.18E1,2,0.0394293,nan
,2016-05-10 07:34:42,0.046 sec,0,.17E1,2,0.0584027,nan
,2016-05-10 07:34:42,0.162 sec,0,.17E1,2,0.0768416,nan
---,---,---,---,---,---,---,---
,2016-05-10 07:34:43,0.884 sec,0,.24E-1,18,0.5176322,nan
,2016-05-10 07:34:43,0.891 sec,0,.23E-1,19,0.5186927,nan
,2016-05-10 07:34:43,0.898 sec,0,.22E-1,19,0.5197327,nan
,2016-05-10 07:34:43,0.905 sec,0,.21E-1,19,0.5207159,nan



See the whole table with table.as_data_frame()


In [14]:
print 'RMSE: {0}'.format(np.sqrt(glm_movie_multi.mse()))
glm_movie_multi_pred = glm_movie_multi.predict(test)
print("GLM predictions: ")
test_headers = ['title'] + exp_multi_X[:]
test_headers.append('rtAllCriticsRating')
df_movie_multi = test[test_headers].cbind(glm_movie_multi_pred)

h2o.export_file(frame=df_movie_multi, path='movie_MULTI_out.csv', force=True)

print df_movie_multi.head(5)


RMSE: 1.05143132847

glm prediction Progress: [##################################################] 100%
GLM predictions: 

Export File Progress: [##################################################] 100%
title                year  country    directorName          Actor1                   MovieStarPowerWCntry    AudienceRating  Genre1    Genre2    Genre3    Actor1NameEndsInA      rtAllCriticsRating    predict
-----------------  ------  ---------  --------------------  ---------------------  ----------------------  ----------------  --------  --------  --------  -------------------  --------------------  ---------
Waiting to Exhale    1995  USA        Forest Whitaker       Whitney Houston                         17.16               3.3  Romance   Comedy    Drama     F                                     5.6    5.52971
Sudden Death         1995  USA        Peter Hyams           Jean-Claude Van Damme                   24.75               2.6  Action    Action    Action    F                 